# 🔢 Getalherkenning met een Neural Network

In dit notebook trainen we een neural network dat **handgeschreven cijfers** kan herkennen.

We gebruiken de beroemde **MNIST dataset** — 70.000 afbeeldingen van cijfers (0-9), elk 8x8 pixels.

### Wat gaan we doen?
1. De afbeeldingen bekijken
2. Begrijpen hoe een afbeelding eruitziet als data
3. Een neural network trainen
4. Testen of het model cijfers correct herkent

---
**Instructie:** Voer iedere cel uit met **Shift+Enter**

## Stap 1: Data laden en bekijken

In [ ]:
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
import numpy as np

# Laad de digits dataset (ingebouwd in scikit-learn, geen download nodig)
digits = load_digits()

print(f"Aantal afbeeldingen: {len(digits.images)}")
print(f"Afbeelding grootte: {digits.images[0].shape[0]}x{digits.images[0].shape[1]} pixels")
print(f"Mogelijke labels: {list(range(10))}")

In [ ]:
# Laten we een aantal voorbeelden bekijken
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
fig.suptitle('Voorbeelden uit de dataset', fontsize=14)

for i, ax in enumerate(axes.flat):
    ax.imshow(digits.images[i], cmap='gray_r')
    ax.set_title(f'Label: {digits.target[i]}', fontsize=12)
    ax.axis('off')

plt.tight_layout()
plt.show()

## Stap 2: Hoe ziet het neural network een afbeelding?

Wij zien een plaatje van een cijfer. Het neural network ziet een **rij getallen** — de helderheid van elke pixel (0 = wit, 16 = zwart).

In [ ]:
# Laten we één afbeelding van dichtbij bekijken
voorbeeld = 0  # Verander dit getal om een andere afbeelding te bekijken

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Links: de afbeelding zoals wij die zien
axes[0].imshow(digits.images[voorbeeld], cmap='gray_r')
axes[0].set_title(f'Wat wij zien (label: {digits.target[voorbeeld]})')
axes[0].axis('off')

# Rechts: de getallen die het neural network ziet
im = axes[1].imshow(digits.images[voorbeeld], cmap='gray_r')
axes[1].set_title('Wat het neural network ziet (pixelwaarden)')
for i in range(8):
    for j in range(8):
        val = int(digits.images[voorbeeld][i, j])
        color = 'white' if val > 8 else 'black'
        axes[1].text(j, i, str(val), ha='center', va='center', fontsize=8, color=color)

plt.tight_layout()
plt.show()

print(f"De afbeelding als rij getallen (64 pixels):")
print(digits.images[voorbeeld].flatten().astype(int))
print(f"\nDeze 64 getallen zijn de INPUT van het neural network.")
print(f"De OUTPUT is een getal van 0-9.")

## Stap 3: Neural network trainen

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Data voorbereiden
X = digits.images.reshape(len(digits.images), -1)  # 8x8 afbeelding -> 64 getallen
y = digits.target

# Schalen
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitsen in train en test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

print(f"Train-set: {len(X_train)} afbeeldingen")
print(f"Test-set:  {len(X_test)} afbeeldingen")

# Neural network aanmaken en trainen
model = MLPClassifier(
    hidden_layer_sizes=(128, 64),  # 2 hidden lagen: 128 en 64 neuronen
    max_iter=300,
    random_state=42
)

print("\nModel wordt getraind...")
model.fit(X_train, y_train)
print("✅ Training voltooid!")

# Training loss visualiseren
plt.figure(figsize=(10, 4))
plt.plot(model.loss_curve_, color='steelblue', linewidth=2)
plt.title('Training Loss (het model wordt steeds beter)')
plt.xlabel('Iteratie')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)
plt.show()

## Stap 4: Resultaten bekijken

In [ ]:
from sklearn.metrics import accuracy_score

# Hoe goed is het model?
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"📊 Accuracy: {accuracy:.1%} van de cijfers wordt correct herkend!")

In [ ]:
# Laten we de voorspellingen visueel bekijken
fig, axes = plt.subplots(3, 5, figsize=(14, 8))
fig.suptitle('Voorspellingen van het Neural Network', fontsize=14)

# Originele afbeeldingen terughalen (voor we ze schaalden)
test_images = scaler.inverse_transform(X_test)

for i, ax in enumerate(axes.flat):
    img = test_images[i].reshape(8, 8)
    ax.imshow(img, cmap='gray_r')
    
    correct = y_pred[i] == y_test[i]
    kleur = 'green' if correct else 'red'
    ax.set_title(f'Voorspeld: {y_pred[i]}  Echt: {y_test[i]}', 
                 color=kleur, fontweight='bold')
    ax.axis('off')

plt.tight_layout()
plt.show()
print("Groen = correct, Rood = fout")

## Stap 5: Confusion Matrix — welke cijfers worden verward?

Sommige cijfers lijken op elkaar (bijv. 3 en 8, of 1 en 7). Laten we kijken waar het model de meeste fouten maakt.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

fig, ax = plt.subplots(figsize=(8, 8))
ConfusionMatrixDisplay.from_predictions(
    y_test, y_pred, ax=ax, cmap='Blues'
)
ax.set_title('Confusion Matrix — Welke cijfers worden verward?')
plt.show()

print("De diagonaal (linksboven → rechtsonder) toont correcte voorspellingen.")
print("Getallen buiten de diagonaal zijn fouten — bijv. een 3 herkend als 8.")

## 🧪 Experimenteer zelf!

Probeer het model aan te passen:

```python
# Kleiner netwerk (minder neuronen):
model = MLPClassifier(hidden_layer_sizes=(16,), max_iter=300, random_state=42)

# Groter netwerk (meer lagen):
model = MLPClassifier(hidden_layer_sizes=(256, 128, 64), max_iter=300, random_state=42)

# Heel klein netwerk (bijna geen capaciteit):
model = MLPClassifier(hidden_layer_sizes=(3,), max_iter=300, random_state=42)
```

Wat verwacht je? Wordt het model beter of slechter met meer of minder neuronen?

In [ ]:
# Experimenteer hier!
model2 = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=300, random_state=42)
model2.fit(X_train, y_train)
y_pred2 = model2.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred2):.1%}")

### Interactieve visualisaties

Bekijk hoe een neural network cijfers herkent in deze interactieve demo's:
- https://dylanlarrabee.me/portfolio/mnist_nn/demo/index.html
- https://www.bulovic.at/cnn/